In [1]:
import openai
import snowflake.connector
from tqdm import tqdm
import os

api_key_file = 'apikey.txt'
if os.path.isfile(api_key_file):
    with open(api_key_file) as f:
        openai.api_key = f.readline()
else:
    print(f"Error: {api_key_file} not found.")

OPENAI_API_KEY = openai.api_key

#login to snowflake db
con = snowflake.connector.connect(user='vishal.kumar@scale.com',
                                 account='pxa65918',
                                 authenticator='externalbrowser',
                                 warehouse='COMPUTE_WH',
                                 database='SCALE_CRAWLER',
                                 role='GENERAL_RO')

cs = con.cursor()

Initiating login request with your identity provider. A browser window should have opened for you to complete the login. If you can't see it, check existing browser windows, or your OS settings. Press CTRL+C to abort and try again...


In [3]:
#Get data from snowflake
sql = f'''
select _ID, 
response:responses[4]:context:candidates[0]:message:content::string as M_Resp_T1_Seal1,
response:responses[4]:context:candidates[1]:message:content::string as M_Resp_T1_Seal2,
response:responses[4]:context:candidates[2]:message:content::string as M_Resp_T1_Seal3,
response:responses[4]:context:selectedId::string as T1_Selected_Response,
response:responses[5]:context:rankedCandidates[0][0].id::string as M_T1_Rank1,
response:responses[5]:context:rankedCandidates[1][0].id::string as M_T1_Rank2,
response:responses[5]:context:rankedCandidates[2][0].id::string as M_T1_Rank3,
response:responses[6]:output::string as Rewritten_1,
response:responses[8]:context:candidates[0]:message:content::string as M_Resp_T2_Seal1,
response:responses[8]:context:candidates[1]:message:content::string as M_Resp_T2_Seal2,
response:responses[8]:context:candidates[2]:message:content::string as M_Resp_T2_Seal3,
response:responses[8]:context:selectedId::string as T2_Selected_Response,
response:responses[9]:context:rankedCandidates[0][0].id::string as M_T2_Rank1,
response:responses[9]:context:rankedCandidates[1][0].id::string as M_T2_Rank2,
response:responses[9]:context:rankedCandidates[2][0].id::string as M_T2_Rank3,
response:responses[10]:output::string as Rewritten_2,
response:responses[12]:context:candidates[0]:message:content::string as M_Resp_T3_Seal1,
response:responses[12]:context:candidates[1]:message:content::string as M_Resp_T3_Seal2,
response:responses[12]:context:candidates[2]:message:content::string as M_Resp_T3_Seal3,
response:responses[12]:context:selectedId::string as T3_Selected_Response,
response:responses[13]:context:rankedCandidates[0][0].id::string as M_T3_Rank1,
response:responses[13]:context:rankedCandidates[1][0].id::string as M_T3_Rank2,
response:responses[13]:context:rankedCandidates[2][0].id::string as M_T3_Rank3,
response:responses[14]:output::string as Rewritten_3,
response:responses[16]:context:candidates[0]:message:content::string as M_Resp_T4_Seal1,
response:responses[16]:context:candidates[1]:message:content::string as M_Resp_T4_Seal2,
response:responses[16]:context:candidates[2]:message:content::string as M_Resp_T4_Seal3,
response:responses[16]:context:selectedId::string as T4_Selected_Response,
response:responses[17]:context:rankedCandidates[0][0].id::string as M_T4_Rank1,
response:responses[17]:context:rankedCandidates[1][0].id::string as M_T4_Rank2,
response:responses[17]:context:rankedCandidates[2][0].id::string as M_T4_Rank3,
response:responses[18]:output::string as Rewritten_4,
response:responses[20]:context:candidates[0]:message:content::string as M_Resp_T5_Seal1,
response:responses[20]:context:candidates[1]:message:content::string as M_Resp_T5_Seal2,
response:responses[20]:context:candidates[2]:message:content::string as M_Resp_T5_Seal3,
response:responses[20]:context:selectedId::string as T5_Selected_Response,
response:responses[21]:context:rankedCandidates[0][0].id::string as M_T5_Rank1,
response:responses[21]:context:rankedCandidates[1][0].id::string as M_T5_Rank2,
response:responses[21]:context:rankedCandidates[2][0].id::string as M_T5_Rank3,
response:responses[22]:output::string as Rewritten_5
from tasks 
where project='6578a7010281a578c59d167a'
and cast(completed_at as Date) > '2024-02-29'
'''
cs.execute(sql)
idf = cs.fetch_pandas_all()


In [8]:
#show idf columns
print(idf.columns)
#show idf number of rows
print(idf.shape[0])
#show idf first 5 rows
print(idf.head())
#deleted idf rows where M_RESP_T1_SEAL1 is None
idf = idf[idf['M_RESP_T1_SEAL1'].notna()]

Index(['_ID', 'M_RESP_T1_SEAL1', 'M_RESP_T1_SEAL2', 'M_RESP_T1_SEAL3',
       'T1_SELECTED_RESPONSE', 'M_T1_RANK1', 'M_T1_RANK2', 'M_T1_RANK3',
       'REWRITTEN_1', 'M_RESP_T2_SEAL1', 'M_RESP_T2_SEAL2', 'M_RESP_T2_SEAL3',
       'T2_SELECTED_RESPONSE', 'M_T2_RANK1', 'M_T2_RANK2', 'M_T2_RANK3',
       'REWRITTEN_2', 'M_RESP_T3_SEAL1', 'M_RESP_T3_SEAL2', 'M_RESP_T3_SEAL3',
       'T3_SELECTED_RESPONSE', 'M_T3_RANK1', 'M_T3_RANK2', 'M_T3_RANK3',
       'REWRITTEN_3', 'M_RESP_T4_SEAL1', 'M_RESP_T4_SEAL2', 'M_RESP_T4_SEAL3',
       'T4_SELECTED_RESPONSE', 'M_T4_RANK1', 'M_T4_RANK2', 'M_T4_RANK3',
       'REWRITTEN_4', 'M_RESP_T5_SEAL1', 'M_RESP_T5_SEAL2', 'M_RESP_T5_SEAL3',
       'T5_SELECTED_RESPONSE', 'M_T5_RANK1', 'M_T5_RANK2', 'M_T5_RANK3',
       'REWRITTEN_5'],
      dtype='object')
967869
                        _ID M_RESP_T1_SEAL1 M_RESP_T1_SEAL2 M_RESP_T1_SEAL3  \
0  65e0c786d7b29b7a9a05ea70            None            None            None   
1  65e0c811189161653834b031           

In [9]:
print(idf.head())

                          _ID  \
179  65d92c03f2f872b87467904f   
348  65e0c6791ce05322af518614   
468  65d9fae62e13b5f3632cd694   
512  65d86d4099ad00066e9d3f6b   
551  65de355941c6ce29ac8d0f6c   

                                       M_RESP_T1_SEAL1  \
179  **Breakdown of the command:** `"icacls . /gran...   
348  Popular cheeses for a charcuterie board includ...   
468  ```python\nfrom tensorflow import keras\nfrom ...   
512  ```sql\nSELECT \n  h.sector,\n  h.control,\n  ...   
551  ## February 26, 2024\n\nUgh. Today was one of ...   

                                       M_RESP_T1_SEAL2 M_RESP_T1_SEAL3  \
179  The command `icacls . /grant Everyone:F /T /C ...            None   
348                                               None            None   
468  ```python\nfrom tensorflow.keras.models import...            None   
512  ```googlesql\nSELECT HD2020.CONTROL, (IC2020.D...            None   
551  ## February 26, 2024 - A Rough Day at the Offi...            None   

    T1_

In [10]:
#idf to csv
idf.to_csv('idf.csv', index=False)                      